In [1]:
import duckdb

## The Question

We landed on this question with Marketing *after* we had a conversation with the Product team. 
<br>
<br>
*The question is:*

"Which demographic segments demonstrate the highest engagement and sales conversion rates for specific product categories, and how can targeted marketing campaigns be optimized to increase these metrics?"
<br>
 <br>
## Next Steps:
1. Load the dataset 'shopping_trends_updated.csv' into a DataFrame.
2. Perform initial data exploration to understand the distribution of demographics and purchase behaviors.
3. Identify the key demographic segments and analyze their engagement and sales conversion rates across different product categories.
4. Use statistical methods to validate the findings.
5. Develop visualizations to represent the insights effectively.
6. Draft recommendations for targeted marketing strategies based on the analyzed data.  
<br>
<br>

# Best Cols to start exploring?

The columns in shopping_trends_updated.csv that can provide a good breakdown of demographic information and help analyze which products perform best within certain demographic segments are:

## Demographic Information:
- Age
- Gender
- Location
- Subscription Status

## Product Performance:
- Item Purchased
- Category
- Purchase Amount (USD)
- Review Rating

These columns can be used to segment the data by demographic details and analyze product preferences and performance within those segments. For example, you can aggregate data to see which product categories are most popular among different age groups or which items have the highest review ratings in different locations.


In [64]:
from pathlib import Path

# Parent goes up to src folder, resolve gets that path, parent again goes to may 2024 folder
data_path = Path().parent.resolve().parent / 'data/shopping_trends_updated.csv'

# Normalize names to make it easier to work with, no spaces
data = duckdb.read_csv(data_path, normalize_names=True)

In [65]:
# Create a table in memory -- 1 way to do it
#con = duckdb.connect(database = ":memory:")
#con.execute('SELECT * FROM SHOPPING_DATA').fetchone()

# Create a table in memory -- another way to do it
duckdb.execute('CREATE TABLE shopping_data as SELECT * FROM data')
# Update the table in memory
data = duckdb.sql('''
           UPDATE shopping_data
           SET frequency_of_purchases = 'EveryTwoWeeks'
           WHERE frequency_of_purchases = 'Fortnightly'
           ''')

CatalogException: Catalog Error: Table with name "shopping_data" already exists!

In [66]:
# Get a high level expectation of the data
duckdb.sql('select * from shopping_data limit 100').show(max_width=1000)

┌─────────────┬───────┬─────────┬────────────────┬─────────────┬─────────────────────┬────────────────┬─────────┬───────────┬─────────┬───────────────┬─────────────────────┬────────────────┬──────────────────┬─────────────────┬────────────────────┬────────────────┬────────────────────────┐
│ customer_id │  age  │ gender  │ item_purchased │  category   │ purchase_amount_usd │   _location    │  size   │   color   │ season  │ review_rating │ subscription_status │ shipping_type  │ discount_applied │ promo_code_used │ previous_purchases │ payment_method │ frequency_of_purchases │
│    int64    │ int64 │ varchar │    varchar     │   varchar   │        int64        │    varchar     │ varchar │  varchar  │ varchar │    double     │       varchar       │    varchar     │     varchar      │     varchar     │       int64        │    varchar     │        varchar         │
├─────────────┼───────┼─────────┼────────────────┼─────────────┼─────────────────────┼────────────────┼─────────┼───────────┼──

In [74]:
duckdb.sql('select distinct promo_code_used, count(*) from shopping_data group by promo_code_used')
# Yes: 1677, No: 2223


┌─────────────────┬──────────────┐
│ promo_code_used │ count_star() │
│     varchar     │    int64     │
├─────────────────┼──────────────┤
│ No              │         2223 │
│ Yes             │         1677 │
└─────────────────┴──────────────┘

In [75]:

duckdb.sql('select distinct payment_method, count(*) from shopping_data group by payment_method')
# Credit Card    │          671 │
# │ Venmo          │          634 │
# │ Cash           │          670 │
# │ Bank Transfer  │          612 │
# │ PayPal         │          677 │
# │ Debit Card     │          636 


┌────────────────┬──────────────┐
│ payment_method │ count_star() │
│    varchar     │    int64     │
├────────────────┼──────────────┤
│ Venmo          │          634 │
│ Bank Transfer  │          612 │
│ Cash           │          670 │
│ Debit Card     │          636 │
│ Credit Card    │          671 │
│ PayPal         │          677 │
└────────────────┴──────────────┘

In [76]:

duckdb.sql('select distinct frequency_of_purchases, count(*) from shopping_data group by frequency_of_purchases')
# ┌────────────────────────┬──────────────┐
# │ frequency_of_purchases │ count_star() │
# │        varchar         │    int64     │
# ├────────────────────────┼──────────────┤
# │ Quarterly              │          563 │
# │ Every 3 Months         │          584 │
# │ Bi-Weekly              │          547 │
# │ Annually               │          572 │
# │ Monthly                │          553 │
# │ Weekly                 │          539 │
# │ EveryTwoWeeks          │          542 │


┌────────────────────────┬──────────────┐
│ frequency_of_purchases │ count_star() │
│        varchar         │    int64     │
├────────────────────────┼──────────────┤
│ Bi-Weekly              │          547 │
│ Annually               │          572 │
│ Monthly                │          553 │
│ Weekly                 │          539 │
│ Quarterly              │          563 │
│ Every 3 Months         │          584 │
│ EveryTwoWeeks          │          542 │
└────────────────────────┴──────────────┘

In [77]:

duckdb.sql('select distinct subscription_status, count(*) from shopping_data group by subscription_status')
# ┌─────────────────────┬──────────────┐
# │ subscription_status │ count_star() │
# │       varchar       │    int64     │
# ├─────────────────────┼──────────────┤
# │ No                  │         2847 │
# │ Yes                 │         1053 │


┌─────────────────────┬──────────────┐
│ subscription_status │ count_star() │
│       varchar       │    int64     │
├─────────────────────┼──────────────┤
│ Yes                 │         1053 │
│ No                  │         2847 │
└─────────────────────┴──────────────┘